# Test utilisation Ray[rllib]

In [2]:
import ray
from ray import tune
import os

c:\Users\p.archipczuk\AppData\Local\anaconda3\envs\RL\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-24 11:05:01,231	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-24 11:05:01,664	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [79]:
ray.init()

2024-06-24 11:41:51,508	INFO worker.py:1761 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.11.7
Ray version:,2.30.0
Dashboard:,http://127.0.0.1:8265


(PPO pid=11880) Error: File '..\data\networks\2way-single-intersection\single-intersection.net.xml' is not accessible (No such file or directory).
(PPO pid=11880) Quitting (on error).
(PPO pid=11880) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=11880, ip=127.0.0.1, actor_id=aa1fe55c022c96af1f42e7e801000000, repr=PPO)
(PPO pid=11880)   File "python\ray\_raylet.pyx", line 1893, in ray._raylet.execute_task
(PPO pid=11880)   File "python\ray\_raylet.pyx", line 1834, in ray._raylet.execute_task.function_executor
(PPO pid=11880)   File "c:\Users\p.archipczuk\AppData\Local\anaconda3\envs\RL\Lib\site-packages\ray\_private\function_manager.py", line 691, in actor_method_executor
(PPO pid=11880)     return method(__ray_actor, *args, **kwargs)
(PPO pid=11880)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(PPO pid=11880)   File "c:\Users\p.archipczuk\AppData\Local\anaconda3\envs\RL\Lib\site-packages\ray\util\tracing\t

## Train our custom environment

In [88]:
from ray.tune.registry import register_env, get_trainable_cls
from pathlib import Path
from gymnasium.spaces import Dict, Discrete, MultiBinary, Box
from ray.rllib.examples.rl_modules.classes.action_masking_rlm
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
import numpy as np
from FpfEnv_v2 import FeuParFeuEnv
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.examples.rl_modules.classes.action_masking_rlm import TorchActionMaskRLM
from ray.rllib.core.rl_module.marl_module import MultiAgentRLModuleSpec, SingleAgentRLModuleSpec
from ray.rllib.algorithms import AlgorithmConfig
from ray.rllib.examples.rl_modules.classes.action_masking_rlm import (
    TorchActionMaskRLM,
)


In [89]:
net_file = r"D:\mesdocuments\paul\gertrude\travail\R&D\projets\generation_diag\travail\data\networks\2way-single-intersection\single-intersection.net.xml"
route_file = r"D:\mesdocuments\paul\gertrude\travail\R&D\projets\generation_diag\travail\data\networks\2way-single-intersection\single-intersection.rou.xml"

base_path = Path(os.path.abspath('')).parent
net_file_path = (base_path / net_file).resolve()
route_file_path = (base_path / route_file).resolve()

register_env('fpf', lambda config : ParallelPettingZooEnv((FeuParFeuEnv(
        net_file=net_file,
        route_file=route_file,
        reward_fn_name='mean_waiting_time',
        with_gui=True,
        num_seconds=1000
))))

In [90]:
base_config = (get_trainable_cls("PPO")
               .get_default_config()
               .environment(
                   env='fpf',
                   action_space = Discrete(3),
                   observation_space = Dict({
                        "observations" : Box(low=0, high=100, shape=(2,), dtype=np.float32),
                        "action_mask" : MultiBinary(3)
                        }
                    ),
                    
                )
               .multi_agent(
                   policies={"p0"},
                   # All agents map to the exact same policy.
                    policy_mapping_fn=(lambda aid, *args, **kwargs: "p0"),
               )
               .training(
                    model={
                        "vf_share_layers": True,
                    },
                    vf_loss_coeff=0.005,
                )
               .rl_module(
                    model_config_dict={
                        "post_fcnet_hiddens": [64, 64],
                        "post_fcnet_activation": "relu",
                    },
                   rl_module_spec = MultiAgentRLModuleSpec(
                       module_specs={"p0" : SingleAgentRLModuleSpec(module_class=TorchActionMaskRLM)},
                   ),
               )
    )

2024-06-24 11:44:18,621	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.rl_module(_enable_rl_module_api=..)` has been deprecated. Use `AlgorithmConfig.api_stack(enable_rl_module_and_learner=..)` instead. This will raise an error in the future!


In [91]:
num_rollout_workers = 5
rollout_fragment_length = 200


config = (
    base_config
    .rollouts(num_rollout_workers=num_rollout_workers, rollout_fragment_length=rollout_fragment_length)
    .framework(framework="torch")
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    .api_stack(enable_rl_module_and_learner=True, enable_env_runner_and_connector_v2 = True)
)

2024-06-24 11:44:19,510	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.env_runners(num_rollout_workers)` has been deprecated. Use `AlgorithmConfig.env_runners(num_env_runners)` instead. This will raise an error in the future!


In [92]:
assert isinstance(config.observation_space, Dict), "Erreur"

In [93]:
# num_rollout_workers = 5
# rollout_fragment_length = 200
# train_batch_size = num_rollout_workers * rollout_fragment_length
# config = (
#         get_trainable_cls('PPO')
#         .environment(env='fpf')
#         .rollouts(num_rollout_workers=num_rollout_workers, rollout_fragment_length=rollout_fragment_length)
#         .training(
#             train_batch_size=train_batch_size)
#         .multi_agent(
#                    policies={"p0"},
#                    # All agents map to the exact same policy.
#                     policy_mapping_fn=(lambda aid, *args, **kwargs: "p0"),
#                )
#         .rl_module(
#                    model_config_dict = {'vf_share_layers' : True},
#                    rl_module_spec = MultiAgentRLModuleSpec(
#                        module_specs={"p0" : SingleAgentRLModuleSpec(module_class=TorchActionMaskRLM)},
#                        marl_module_class=TorchActionMaskRLM,
#                    ),
#                )
#         .experimental(_disable_preprocessor_api=True)
#         .debugging(log_level="ERROR")
#         .framework(framework="torch")
#         .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
#     )


In [94]:
# num_rollout_workers = 5
# rollout_fragment_length = 200
# train_batch_size = num_rollout_workers * rollout_fragment_length
# config = (
#         PPOConfig()
#         .environment(env='fpf')
#         .rollouts(num_rollout_workers=num_rollout_workers, rollout_fragment_length=rollout_fragment_length)
#         .training(
#             train_batch_size=train_batch_size)
#         .experimental(_disable_preprocessor_api=True)
#         .debugging(log_level="ERROR")
#         .framework(framework="torch")
#         .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
#     )

In [95]:
tune.run(
        "PPO",
        name="PPO",
        stop={"num_episodes": 1},
        checkpoint_freq=100,
        storage_path=r"D:\mesdocuments\paul\gertrude\travail\R&D\projets\generation_diag\travail\outputs\fpf",
        config=config.to_dict(),
    )

2024-06-24 11:44:23,387	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2024-06-24 11:44:23,389	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_cpus_per_worker` has been deprecated. Use `AlgorithmConfig.num_cpus_per_env_runner` instead. This will raise an error in the future!
2024-06-24 11:44:23,389	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_gpus_per_worker` has been deprecated. Use `AlgorithmConfig.num_gpus_per_env_runner` instead. This will raise an error in the future!
2024-06-24 11:44:23,389	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_learner_workers` has been deprecated. Use `AlgorithmConfig.num_learners` instead. This will raise an error in the future!
2024-06-24 11:44:23,389	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_cpus_per_learner_worker` has been deprecated. Use `AlgorithmConfig.num_cpus_per_learner` instead. This will raise an error in the future!
2024-06-24 11:44:23,389	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_gpus

(MultiAgentEnvRunner pid=46056) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 2ms, vehicles TOT 0 ACT 0 BUF 0)                      
(PPO pid=27712) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 5x across cluster]


Trial name
PPO_fpf_567f6_00000


2024-06-24 11:44:38,852	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'D:/mesdocuments/paul/gertrude/travail/R&D/projets/generation_diag/travail/outputs/fpf/PPO' in 0.0007s.


(PPO pid=27712) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=27712, ip=127.0.0.1, actor_id=9e4d008ed6c0e0482d3eac4e01000000, repr=PPO)
(PPO pid=27712)   File "python\ray\_raylet.pyx", line 1893, in ray._raylet.execute_task
(PPO pid=27712)   File "python\ray\_raylet.pyx", line 1834, in ray._raylet.execute_task.function_executor
(PPO pid=27712)   File "c:\Users\p.archipczuk\AppData\Local\anaconda3\envs\RL\Lib\site-packages\ray\_private\function_manager.py", line 691, in actor_method_executor
(PPO pid=27712)     return method(__ray_actor, *args, **kwargs)
(PPO pid=27712)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(PPO pid=27712)   File "c:\Users\p.archipczuk\AppData\Local\anaconda3\envs\RL\Lib\site-packages\ray\util\tracing\tracing_helper.py", line 467, in _resume_span
(PPO pid=27712)     return method(self, *_args, **_kwargs)
(PPO pid=27712)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(PPO pid=27712)   Fil

TuneError: ('Trials did not complete', [PPO_fpf_567f6_00000])

## Test for Custom env

In [ ]:
net_file = r"..\data\networks\2way-single-intersection\single-intersection.net.xml"
route_file = r"..\data\networks\2way-single-intersection\single-intersection.rou.xml"

base_path = Path(os.path.abspath('')).parent
net_file_path = (base_path / net_file).resolve()
route_file_path = (base_path / route_file).resolve()

env = FeuParFeuEnv(
        net_file=net_file,
        route_file=route_file,
        reward_fn_name='green',
        with_gui=True,
        num_seconds=3600
)

for _ in range(1):
    obs, infos = env.reset()
    while True:
        actions = {agent: env.action_space(agent).sample(obs['action_mask']) for agent in env.agents}
        observations, rewards, terminations, truncations, infos = env.step(actions)
        if all(terminations.values()) or any(truncations.values()):
            break

In [48]:
import numpy as np

class Test():
    def __init__(self) -> None:
        print("init")
        self.possible_actions = ['g', 'y', 'r']
        self.consecutive_durations = {state : 0 for state in self.possible_actions}
        self.cumulated_durations = {state : 0 for state in self.possible_actions}  
        print(f"self.consecutive_durations : {self.consecutive_durations}") 
        print(f"self.cumulated_durations : {self.cumulated_durations}")

    def update(self, state: str):
        """"Update all the values related to this Feu such as cumulated green time etc
        Args:
            state (str): should be 'g', 'y' or 'r' 
        """
        print("----------------UPDATE-----------------")
        new = {key : value + 1 if key == state else 0 for key, value in self.consecutive_durations.items()}
        self.consecutive_durations.update(new)
        self.cumulated_durations[state] += 1
        print(f"new self.consecutive_durations : {self.consecutive_durations} ")
        print(f"new self.cumulated_durations : {self.cumulated_durations} ")

    def compute_action_mask(self, state) -> np.array:
        """Return the action mask for the Feu.
        Should be a list or array with
        1 : valid action
        0 : invalid action
        """
        print("----------------COMPUTE ACTION MASK-----------------")
        current_state = state
        print(f"current_state : {current_state}")
        print(f"self.consecutive_durations : {self.consecutive_durations}")
        if current_state == "g":
            # ctr vert min
            if self.consecutive_durations["g"] < 6:
                action_mask = np.array([1, 0, 0], dtype=np.int8)
            else:
                action_mask = np.array([1, 1, 0], dtype=np.int8)
        elif current_state == "y":
            # ctr orange 
            if self.consecutive_durations["y"] < 3:
                action_mask = np.array([0, 1, 0], dtype=np.int8)
            elif self.consecutive_durations["y"] == 3:
                action_mask = np.array([0, 0, 1], dtype=np.int8)
            else:
                raise Exception(f"Impossible d'avoir plus de 3 sec de orange.\nself.consecutive_durations : {self.consecutive_durations['y']}")
                #action_mask = np.array([0, 0, 1], dtype=np.int8)
        else:
            # ctr rouge max
            if self.consecutive_durations["r"] >= 120:
                action_mask = np.array([1, 0, 0], dtype=np.int8)
            else:
                action_mask = np.array([1, 0, 1], dtype=np.int8)
        print(f"action_mask : {action_mask}")
        return action_mask
    
    def process(self, state: str):
        self.update(state=state)
        self.compute_action_mask(state=state)


In [81]:
t = Test()

init
self.consecutive_durations : {'g': 0, 'y': 0, 'r': 0}
self.cumulated_durations : {'g': 0, 'y': 0, 'r': 0}


In [87]:
t.process(state='y')

----------------UPDATE-----------------
new self.consecutive_durations : {'g': 6, 'y': 0, 'r': 0} 
new self.cumulated_durations : {'g': 6, 'y': 0, 'r': 0} 
----------------COMPUTE ACTION MASK-----------------
current_state : g
self.consecutive_durations : {'g': 6, 'y': 0, 'r': 0}
action_mask : [1 1 0]


In [2]:
import numpy as np

In [6]:
action = np.random.choice(np.array([0, 2]))
action

2